To make learning faster, we'll introduce batch normalization for RNN.

I refered to the following paper:
https://arxiv.org/pdf/1603.09025.pdf

In [1]:
!pip install matplotlib --upgrade
!pip install quandl --upgrade
!pip install yahoo-finance --upgrade

#generic
%matplotlib inline
from __future__ import print_function

import numpy as np
import time
import quandl 
import matplotlib.pyplot as plt

from yahoo_finance import Share



def get_data_by_key(key, data):
    data_it = iter(data)
    return_data = []
    flag = True
    for d in data_it:
        return_data.append(d[key])
        
    return np.array(return_data)

def get_data_by_list(name_list, start_date, end_date, data_type="Open"):
    share_list = []
    new_name_list = []
    for name in name_list:
        try:
            share_list.append(Share(name))
            new_name_list.append(name)
        except:
            pass
    
    stock_data_list = []
    date = []
    flag = True
    N_data = 0
    fail_name_list = []
    ret_name_list = []
    for idx, share in enumerate(share_list):
        name = new_name_list[idx]
        try:
            hist_data = share.get_historical(start_date=start_date, end_date=end_date)
            stock_data = map(float, get_data_by_key(key=data_type, data=hist_data))
            n_data = len(stock_data)
            if n_data == 0:
                fail_name_list.append(name)
            date.append(get_data_by_key(key='Date', data=hist_data))
            stock_data_list.append(stock_data)
            ret_name_list.append(name)
        except:
            pass
    print ("fail_name_list: ", fail_name_list)
    return np.array(stock_data_list).T, date, ret_name_list

Requirement already up-to-date: matplotlib in /usr/local/lib/python2.7/dist-packages
Requirement already up-to-date: numpy>=1.6 in /usr/local/lib/python2.7/dist-packages (from matplotlib)
Requirement already up-to-date: python-dateutil in /usr/local/lib/python2.7/dist-packages (from matplotlib)
Requirement already up-to-date: pytz in /usr/local/lib/python2.7/dist-packages (from matplotlib)
Requirement already up-to-date: cycler in /usr/local/lib/python2.7/dist-packages (from matplotlib)
Requirement already up-to-date: pyparsing!=2.0.4,>=1.5.6 in /usr/local/lib/python2.7/dist-packages (from matplotlib)
Cleaning up...
Requirement already up-to-date: quandl in /usr/local/lib/python2.7/dist-packages
Cleaning up...
Requirement already up-to-date: yahoo-finance in /usr/local/lib/python2.7/dist-packages
Requirement already up-to-date: pytz in /usr/local/lib/python2.7/dist-packages (from yahoo-finance)
Requirement already up-to-date: simplejson in /usr/local/lib/python2.7/dist-packages (from y

In [35]:
energy = ["PTR", "XOM", "CVX", "RDS-A", "BP", "TOT", "SLB", "KMI", "COP", "CEO", 
          "E", "STO", "OXY", "PBR", "EOG", "APC", "SU", "ENB", "HAL", "WMB"]
financial = ["WFC", "JPM", "HSBC", "BAC", "C", "SAN", "MTU", "RY", "WBK", "TD", "GS", 
             "LYG", "AXP", "AIG", "MS", "ITUB", "BCS", "BBVA"]
healthcare = ["NVS", "JNJ", "PFE", "MRK", "GILD", "SNY", "AMGN", "NVO", "GSK", "UNH",
             "MDT", "BMY", "CELG", "BIIB", "AZN", "LLY", "ABT", "AGN", "VRX", "TEVA",
             "TMO", "SHPG", "REGN"]
buisiness = ["ACN", "LMT", "CNI", "FDX", "DAL", "CSX", "AAL", "CP", "NSC", "NOC", "LUV"]
telecom = ["CHL", "VZ", "T", "VOD", "NTT", "AMX", "CHA", "BT", "CHU", "ORAN", "BCE",
           "CHT", "SKM", "TI", "TU", "S", "TLK", "DUK", "NGG", "NEE", "D", "SO", "EXC", "KEP",
          "AEP", "SRE", "PCG", "HNP", "PPL", "PEG", "EIX", "ETP", "ED", "ENI", "XEL", "ES", "FE"]
hardware = ["AAPL", "ORCL", "IBM", "INTC", "CSCO", "TSM", "QCOM", "HPQ", "TXN", "EMC",
           "CAJ", "ASML", "ERIC", "SNE", "AVGO", "MU", "GLW", "NXPI", "NOK",
           "AMAT", "WDC", "WIT", "ADI", "STX", "APH"]
software = ["MSFT", "GOOGL", "BIDU", "EBAY", "SAP", "CRM", "YHOO", "VMW",
            "ADBE", "CTSH", "INFY", "INTU", "LNKD", "RHT", "NTES", "CHKP", 
            "CA", "ADSK", "AKAM", "NVDA"]
industrial = ["GE", "V", "MA", "UTX", "MMM", "BA", "UNP", "UPS", "HON", "DHR", "CAT", 
              "ABB", "GD", "ADP", "EMR", "ITW", "ECL", "TEL", "PCAR", "WM"]
manufacturing = ["TM", "HMC", "F", "GM", "JCI", "TSLA", "TTM", "CMI", "DLPH", "MGA",
                "CMI", "DLPH", "MGA", "GPC", "BWA", "HOG", "ALV", "HAR", "LEA", "LKQ", 
                 "WBC", "GT", "FCAU", "NSANY", "NAV"]
consumer = ["PG", "BUD", "KO", "PEP", "UL", "PM", "BTI", "MO", "ABEV", "DEO", "MDLZ",
            "CL", "MON", "MCK", "KMB", "WHR", "DIS", "CMCSA", "FOXA", "TWX", "TWC",
           "DISH", "CBS", "DISCA", "TV", "CHTR", "QVCA", "OMC", "NLSN", 
           "PSO", "SJR", "NFLX"]
diversified = ["BRK-A", "BRK-B", "UTX", "BC", "RTN", "OLN", "ITT", "MSBHY", "KWHIY", 
               "ABB", "IEP", "GE"]
retailing = ["WMT", "AMZN", "HD", "MCD", "NKE", "SBUX", "CVS", "WBA", "PCLN", "COST", 
            "TGT", "TJX", "LVS", "YUM", "CCL", "LUX", "DG", "M", "AZO", "ROST", "CMG", "GPS", 
            "DLTR", "RCL", "KSS", "HOT", "JWN"]

input_list = energy + financial + healthcare + buisiness + telecom\
+ hardware + software + industrial + manufacturing + consumer + diversified + retailing
print (len(input_list))

270


In [7]:
import time
print ("Started!!")

st = time.time()
start_date="2014-04-01"
end_date="2016-04-01"
input_data, date, input_list = get_data_by_list(input_list, start_date=start_date, end_date=end_date) 
sp = quandl.get("YAHOO/INDEX_GSPC", start_date=start_date, end_date=end_date)
target_data = np.array(sp['Open'])
elapsed = time.time() - st
print ("time for getting training_data:", elapsed)

Started!!
fail_name_list:  ['NEE']
time for getting training_data: 1020.07868218


In [36]:
index = input_list.index("NEE")
print (input_list[index])
print (input_data[index])

NEE
[]


In [37]:
input_data_tilde = np.delete(input_data, index, 0)

In [38]:
length = len(input_data_tilde[0])
for i in xrange(len(input_data_tilde)):
    if length != len(input_data_tilde[i]):
        print (i)
        
data_list = []
for data in iter(input_data_tilde):
    data_list.append(data)
data = np.array(data_list)
print (data.shape)

(269, 505)


In [39]:
input_data = data.T

In [40]:
energy = ["PTR", "XOM", "CVX", "RDS-A", "BP", "TOT", "SLB", "KMI", "COP", "CEO", 
          "E", "STO", "OXY", "PBR", "EOG", "APC", "SU", "ENB", "HAL", "WMB"]
financial = ["WFC", "JPM", "HSBC", "BAC", "C", "SAN", "MTU", "RY", "WBK", "TD", "GS", 
             "LYG", "AXP", "AIG", "MS", "ITUB", "BCS", "BBVA"]
healthcare = ["NVS", "JNJ", "PFE", "MRK", "GILD", "SNY", "AMGN", "NVO", "GSK", "UNH",
             "MDT", "BMY", "CELG", "BIIB", "AZN", "LLY", "ABT", "AGN", "VRX", "TEVA",
             "TMO", "SHPG", "REGN"]
buisiness = ["ACN", "LMT", "CNI", "FDX", "DAL", "CSX", "AAL", "CP", "NSC", "NOC", "LUV"]
telecom = ["CHL", "VZ", "T", "VOD", "NTT", "AMX", "CHA", "BT", "CHU", "ORAN", "BCE",
           "CHT", "SKM", "TI", "TU", "S", "TLK", "DUK", "NGG", "D", "SO", "EXC", "KEP",
          "AEP", "SRE", "PCG", "HNP", "PPL", "PEG", "EIX", "ETP", "ED", "ENI", "XEL", "ES", "FE"]
hardware = ["AAPL", "ORCL", "IBM", "INTC", "CSCO", "TSM", "QCOM", "HPQ", "TXN", "EMC",
           "CAJ", "ASML", "ERIC", "SNE", "AVGO", "MU", "GLW", "NXPI", "NOK",
           "AMAT", "WDC", "WIT", "ADI", "STX", "APH"]
software = ["MSFT", "GOOGL", "BIDU", "EBAY", "SAP", "CRM", "YHOO", "VMW",
            "ADBE", "CTSH", "INFY", "INTU", "LNKD", "RHT", "NTES", "CHKP", 
            "CA", "ADSK", "AKAM", "NVDA"]
industrial = ["GE", "V", "MA", "UTX", "MMM", "BA", "UNP", "UPS", "HON", "DHR", "CAT", 
              "ABB", "GD", "ADP", "EMR", "ITW", "ECL", "TEL", "PCAR", "WM"]
manufacturing = ["TM", "HMC", "F", "GM", "JCI", "TSLA", "TTM", "CMI", "DLPH", "MGA",
                "CMI", "DLPH", "MGA", "GPC", "BWA", "HOG", "ALV", "HAR", "LEA", "LKQ", 
                 "WBC", "GT", "FCAU", "NSANY", "NAV"]
consumer = ["PG", "BUD", "KO", "PEP", "UL", "PM", "BTI", "MO", "ABEV", "DEO", "MDLZ",
            "CL", "MON", "MCK", "KMB", "WHR", "DIS", "CMCSA", "FOXA", "TWX", "TWC",
           "DISH", "CBS", "DISCA", "TV", "CHTR", "QVCA", "OMC", "NLSN", 
           "PSO", "SJR", "NFLX"]
diversified = ["BRK-A", "BRK-B", "UTX", "BC", "RTN", "OLN", "ITT", "MSBHY", "KWHIY", 
               "ABB", "IEP", "GE"]
retailing = ["WMT", "AMZN", "HD", "MCD", "NKE", "SBUX", "CVS", "WBA", "PCLN", "COST", 
            "TGT", "TJX", "LVS", "YUM", "CCL", "LUX", "DG", "M", "AZO", "ROST", "CMG", "GPS", 
            "DLTR", "RCL", "KSS", "HOT", "JWN"]

input_list = energy + financial + healthcare + buisiness + telecom\
+ hardware + software + industrial + manufacturing + consumer + diversified + retailing
print (len(input_list))

269


In [30]:
target_data = np.atleast_2d(target_data).T
print (target_data.shape)

(505, 1)


In [41]:
df_input = input_data[1:] / input_data[:-1] - 1.0
df_target = target_data[1:] / target_data[:-1] -1.0

In [10]:
import tensorflow as tf

In [47]:
class BNLSTM(object):
    
    def __init__(self, layers, activation=tf.sigmoid, is_initialize=True):
        """Initialize Vanilla LSTM with Batch Normalization
        
        Args:
            layers (List(int)): each element should be the number of components
            is_initialized(bool): if this variable is True, glaph will be initialized
        """
        self.layers = layers
        self.activation = activation
        
        if is_initialize is True:
            sess =tf.InteractiveSession()
            tf.reset_default_graph()
            sess.close()  
        
    def weight_variable(self, shape):
        initial = tf.truncated_normal(shape, stddev=1.0)
        return tf.Variable(initial, name="weight")
        
    def bias_variable(self, shape):
        initial = tf.constant(0.0, shape=shape)
        return tf.Variable(initial, name="bias")
        
    def batch_normalization(self, input, shape):
        # input should be hidden_dim
        eps = 1e-5
        gamma = tf.Variable(tf.constant(1.0, shape=shape))
        beta = tf.Variable(tf.constant(0.0, shape=shape))
        mean, variance = tf.nn.moments(input, [0])
        return gamma * (input - mean) / tf.sqrt(variance + eps) + beta
    
    def build_model(self, n_split):
        # build model for trainig
        self._input = tf.placeholder(tf.float32, [None, self.layers[0]])
        self._target = tf.placeholder(tf.float32, [None, self.layers[-1]])
        
        # when learning, we will split data which has shape (T, dim)
        # into (n_batch, dim) * T / n_batch
        input_tilde = tf.unpack(tf.split(0, n_split, self._input))
        target_tilde = tf.unpack(tf.split(0, n_split, self._target))
        
        # the name of variable scope will be layer0, layer1, ...
        x = tf.concat(0, input_tilde)
        for i_layer in xrange(len(self.layers) - 1):
            with tf.variable_scope("layer%d" % i_layer):
                # build LSTM graph for each layer
                self._cell = tf.nn.rnn_cell.BasicLSTMCell(self.layers[i_layer + 1])
                self._initial_state = self._cell.zero_state(1, tf.float32)
                cell_outputs, state = tf.nn.rnn(cell=self._cell, 
                                                                      inputs=input_tilde, 
                                                                      initial_state=self._initial_state)
                self._final_state = state
                n_in = self.layers[i_layer]
                n_out = self.layers[i_layer + 1]
                W = self.weight_variable([n_in, n_out])
                b = self.bias_variable([n_out])
                z = tf.matmul(x, W) + b
            # we will not apply activation function for the last layer
            if i_layer == len(self.layers) - 2:
                self._output = z
            else:
                shape = [self.layers[i_layer + 1]]
                normalized_z = self.batch_normalization(z, shape)
                x = self.activation(normalized_z)
        
        # training graph
        # self._learning_rate = tf.placeholder(tf.float32, shape=[], name="learning rate")
        self.loss = tf.reduce_mean(tf.square(self._output - self._target))
        # optimizer = tf.train.AdamOptimizer(self._learning_rate)
        optimizer = tf.train.AdamOptimizer()
        self.train = optimizer.minimize(self.loss) 
    
    def training(self, input_data, target_data, n_batch=20, n_epochs=100,
                         learning_rate=1e-8, decay_rate=0.5, decay_freq=100):
        n_data = len(input_data)
        print_freq = int(n_epochs / 10)
        
        # build training graph
        n_split = int(n_data / n_batch)
        self.build_model(n_split)
        
        # we'll use only recent n_batch * n_split data 
        # because of garph's architechture
        input_data = input_data[n_data - n_batch * n_split:]
        target_data = input_data[n_data - n_batch * n_split:]
        
        # Add ops to save and restore all the variables.
        saver = tf.train.Saver()
        init_op = tf.initialize_all_variables()
        with tf.Session() as sess:
            sess.run(init_op)
            n_iter = int(n_data / n_batch)
            batch_index = np.arange(n_iter - 1)   # index of data for shuffling
            for epoch in xrange(n_epochs):
                """
                sess.run(self.train, 
                                feed_dict={self._input: input_data,
                                                     self._target: target_data,
                                                     self._learning_rate: learning_rate})
                """
                sess.run(self.train, 
                                feed_dict={self._input: input_data,
                                                     self._target: target_data})
                if (epoch + 1) % decay_freq == 0:
                    learning_rate *= decay_rate
                    print ("change learning rate", learning_rate)
                if epoch % print_freq == 0:
                    print ("loss:", 
                               self.loss.eval(session=sess, 
                                                      feed_dict={self._input: input_data,
                                                                           self._target: target_data}))     
            save_path = saver.save(sess, "/jupyter/tomoaki/model.ckpt")
            print("Model saved in file: %s" % save_path)
            
    def predict(self, input_data):
        saver = tf.train.Saver()
        with  tf.Session() as sess:
            # sess.run(self.init_op)
            # Restore variables from disk.
            saver.restore(sess, "/jupyter/tomoaki/model.ckpt")
            print("Model restored.")
            output = self._output.eval(session=sess,
                                                           feed_dict={self._input: input_data})
        return output    

In [48]:
N = int(len(df_input) * 0.5)

input_train = df_input[:N]
target_train = df_target[:N]
input_pred = df_input[N:]
target_pred = df_target[N:]
n_in = len(input_train[0])
n_hidden1 = int(0.3 * n_in)
n_out = 1
n_batch=20

lstm = BNLSTM(layers=[n_in, n_hidden1, n_out], is_initialize=True)

print ("start!")
st_s = time.time()
lstm.training(input_train, target_train,
                      learning_rate=0.1, decay_freq=100, decay_rate=0.5,
                      n_batch=n_batch, n_epochs=1000)
elapsed_s = time.time() - st_s
print ("learning_time:",  elapsed_s)

pred = mlp.predict(input_pred)
plt.plot(pred, label="predit")
plt.plot(target_pred, label="raw data")
plt.ylim(min(target_pred), max(target_pred))

plt.legend()

start!


ValueError: Cannot feed value of shape (228, 269) for Tensor u'Placeholder_1:0', which has shape '(?, 1)'